# 超级简单的初次爬虫尝试
    为了数据库大作业搞点数据
    - 模仿豆瓣爬虫教程：https://www.bilibili.com/video/BV12E411A7ZQ?share_source=copy_web

    - 目标网站：https://www.cnhnb.com/supply/

In [1]:
from base64 import decode
import bs4
import re
import urllib.request
import urllib.error
import urllib.parse
import xlwt
import sqlite3

In [2]:
def askURL(url):
    '''
    得到一个指定的URL的页面内容
    '''
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:99.0) Gecko/20100101 Firefox/99.0"
    }
    req = urllib.request.Request(url=url,headers=headers)
    try:
        response = urllib.request.urlopen(req)
        html=response.read().decode("utf-8")
        
    except urllib.error.URLError as e:
        if hasattr(e,"code"):
            print(e.code)
        if hasattr(e,"reason"):
            print(e.reason)
    return html

    - 图片好像是动态加载/懒加载？的 不知道怎么爬 以后再学

In [3]:
#商品图片
# findImageLink = re.compile(r'<img.*src="(.*?)"',re.S) 

findtitle = re.compile(r'<img alt="(.*)" data-v-71cf4cda="" title="',re.S)
findprice = re.compile(r'<span class="sp1" data-v-71cf4cda="">(.*)</span>')
findlocation = re.compile(r'<div class="r-shop-btm" data-v-71cf4cda="">(.*?)</div>')
findshop = re.compile(r'<div class="l-shop-btm" data-v-71cf4cda="">(.*?)</div>')
findturnover = re.compile(r'<div class="turnover" data-v-71cf4cda="">(.*?)</div>',re.S)

In [4]:
def getData(html):
    datalist=[]
    soup =bs4.BeautifulSoup(html,"html.parser")
    for item in soup.find_all('div',class_="show-ctn"): #查找网页显示的列表中的一个item
        # print(item)
        data = []
        
        item = str(item) 

        # imagelink = re.findall(findImageLink,item)[0]
        # data.append(imagelink)  

        title =re.findall(findtitle,item)[0]
        data.append(title)

        price = re.findall(findprice,item)[0]
        data.append(price)  

        location = re.findall(findlocation,item)[0]
        data.append(location)  

        shop = re.findall(findshop,item)[0]
        data.append(shop) 

        turnover = re.findall(findturnover,item)[0].replace('\n', '').strip()
        data.append(turnover)

        datalist.append(data)
    return datalist

In [5]:
def saveData(datalist,savepath):
    '''
    保存数据到文件savepath
    '''
    print("save...")
    book = xlwt.Workbook(encoding="utf-8",style_compression=0)
    sheet = book.add_sheet('水果',cell_overwrite_ok=True)
    col = ("title","price","location","shop","turnpver")
    for i in range(0,5):
        sheet.write(0,i,col[i])
    for i in range(len(datalist)):
        data=datalist[i]
        for j in range(0,5):
            sheet.write(i+1,j,data[j])
    book.save(savepath)

In [6]:
html=askURL("https://www.cnhnb.com/p/sgzw/")
datalist=getData(html)

savepath = "农产品.xls"
saveData(datalist,savepath)


save...
